# Mesh Conversion

Converting meshes to other files formats

- we need npz arrays of incomplete meshes
- these are colored point clouds stored as RGBS arrays

1) Load the trimesh scene
2) sample each submesh into a separate pointcloud
3) Combine them into one big material pointcloud
4) Resample to theset number of points
5) cut random holes in the point cloud by selecting nearest neighbours
6) Save the complete sampled shape as a npz file
    > id/id_normalized_color_samples100000_bbox-0.8,0.8,-0.15,2.1,-0.8,0.8.npz
7) save 4 variations in the same folder

## Loading the mesh

In [1]:
import trimesh
from scipy.spatial import cKDTree as KDTree
import numpy as np
import os
meshPath = "C:/Users/u0146408/Documents/Datasets/Shapenet/03001627/1b5e876f3559c231532a8e162f399205/models/model_normalized.obj"

mesh = trimesh.load(meshPath)

In [2]:
mesh.show()

### Scenes

When a mesh has multiple materials and is exported to a obj, the mesh gets subdivided in multiple submeshes. Trimesh reads this as a scene. In order to convert it to a mesh, we need to "dump it"<sup>tm</sup>

In [56]:
print("mesh object:")
print(mesh)
# all the ids of the nodes that have geometry attached to them (no cameras or lights)
print("the geometry nodes:")
print(mesh.graph.nodes_geometry)
# the hash values linked to the identifiers
print("the geometry identifiers:")
print(mesh.geometry_identifiers)


mesh object:
<trimesh.Scene(len(geometry)=3)>
the geometry nodes:
['material_2_24', 'material_1_24', 'material_0_1_8']
the geometry identifiers:
{'ff1173ad8c353141424418c017a1b059d9123af07a1f15cbb98d650805102bb8': 'material_2_24', 'efd8a7407f0b171cebd660c1d152db1b72210e4fd8a46631256fc676cb7954a9': 'material_1_24', '6692102fe8b3c2cf49532f3c9724668fa36080ef6dac07a802bdd35237fce81e': 'material_0_1_8'}


In [3]:
def as_mesh(scene_or_mesh):
    if isinstance(scene_or_mesh, trimesh.Scene):
        mesh = trimesh.util.concatenate([
            trimesh.Trimesh(vertices=m.vertices, faces=m.faces)
            for m in scene_or_mesh.geometry.values()])
    else:
        mesh = scene_or_mesh
    return mesh

def create_grid_points_from_xyz_bounds(min_x, max_x, min_y, max_y ,min_z, max_z, res):
    x = np.linspace(min_x, max_x, res)
    y = np.linspace(min_y, max_y, res)
    z = np.linspace(min_z, max_z, res)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij', sparse=False)
    X = X.reshape((np.prod(X.shape),))
    Y = Y.reshape((np.prod(Y.shape),))
    Z = Z.reshape((np.prod(Z.shape),))

    points_list = np.column_stack((X, Y, Z))
    del X, Y, Z, x
    return points_list

import numbers

def shoot_holes(vertices, n_holes, dropout, mask_faces=None, faces=None,
                rng=None):
    """Generate a partial shape by cutting holes of random location and size.

    Each hole is created by selecting a random point as the center and removing
    the k nearest-neighboring points around it.

    Args:
        vertices: The array of vertices of the mesh.
        n_holes (int or (int, int)): Number of holes to create, or bounds from
            which to randomly draw the number of holes.
        dropout (float or (float, float)): Proportion of points (with respect
            to the total number of points) in each hole, or bounds from which
            to randomly draw the proportions (a different proportion is drawn
            for each hole).
        mask_faces: A boolean mask on the faces. 1 to keep, 0 to ignore. If
                    set, the centers of the holes are sampled only on the
                    non-masked regions.
        faces: The array of faces of the mesh. Required only when `mask_faces`
               is set.
        rng: (optional) An initialised np.random.Generator object. If None, a
             default Generator is created.

    Returns:
        array: Indices of the points defining the holes.
    """
    if rng is None:
        rng = np.random.default_rng()

    if not isinstance(n_holes, numbers.Integral):
        n_holes_min, n_holes_max = n_holes
        n_holes = rng.integers(n_holes_min, n_holes_max)

    if mask_faces is not None:
        valid_vertex_indices = np.unique(faces[mask_faces > 0])
        valid_vertices = vertices[valid_vertex_indices]
    else:
        valid_vertices = vertices

    # Select random hole centers.
    center_indices = rng.choice(len(valid_vertices), size=n_holes)
    centers = valid_vertices[center_indices]

    n_vertices = len(valid_vertices)
    if isinstance(dropout, numbers.Number):
        hole_size = n_vertices * dropout
        hole_sizes = [hole_size] * n_holes
    else:
        hole_size_bounds = n_vertices * np.asarray(dropout)
        hole_sizes = rng.integers(*hole_size_bounds, size=n_holes)

    # Identify the points indices making up the holes.
    kdtree = KDTree(vertices, leafsize=200)
    to_crop = []
    for center, size in zip(centers, hole_sizes):
        _, indices = kdtree.query(center, k=size)
        to_crop.append(indices)
    to_crop = np.unique(np.concatenate(to_crop))
    return to_crop


In [4]:
# shorthands
bbox = [-7, 7, -1, 20, -7, 7]
res = 128
num_points = 100000
bbox_str = str(bbox)
grid_points = create_grid_points_from_xyz_bounds(*bbox, res)
kdtree = KDTree(grid_points)

## Point sampling

converts a trimesh obj to a compressed sampled pointcloud, including the indexes of each material as a color

In [5]:
import trimesh
from scipy.spatial import cKDTree as KDTree
import numpy as np
import os
from pathlib import Path

class meshArray ():

    def __init__(self, R = None, G = None, B = None, S = None, pcd = None, bbox = None, res = None):
        self.R = R
        self.G = G
        self.B = B
        self.S = S
        self.pcd = pcd
        self.bbox = bbox
        self.res = res

        self.fullPcd = None
        self.id = None
    
    def from_trimesh(self, meshPath):

        mesh = trimesh.load(meshPath)
        # check if the mesh multiple materials (sub meshes)
        if isinstance(mesh, trimesh.Scene):
            pointArray = []
            i = 1
            subMeshes = mesh.dump()
            for sub in subMeshes:
                colored_addition = np.hstack((sub.sample(num_points), np.ones((num_points,1)) * i))
                pointArray.append(colored_addition)
                i+=1

            pointArray = np.asarray(pointArray).reshape((-1,4))
            colored_point_cloud = pointArray[np.random.choice(len(pointArray), size=num_points, replace=False)]
        else:
            colored_point_cloud = np.hstack((mesh.sample(num_points), np.ones((num_points,1)))) 

        # encode uncolorized, complete shape of object (at inference time obtained from IF-Nets surface reconstruction)
        # encoding is done by sampling a pointcloud and voxelizing it (into discrete grid for 3D CNN usage)
        full_shape = as_mesh(mesh)
        shape_point_cloud = full_shape.sample(num_points)
        S = np.zeros(len(grid_points), dtype=np.int8)
        self.fullPcd = shape_point_cloud

        _, idx = kdtree.query(shape_point_cloud)
        S[idx] = 1

        self.S = S
        self.R = self.G = self.B =  colored_point_cloud[:,3]
        self.pcd = colored_point_cloud[:,:3]

    def filter_points(self, outPath, nrOfVariants, nrOfHoles, dropout):
        
        for nr in range(nrOfVariants):
        
            filteredIndexes = shoot_holes(self.pcd, nrOfHoles, dropout)
            filteredPcd = np.delete(self.pcd,filteredIndexes,0)
            filteredColor = np.delete(self.R,filteredIndexes,0)
            np.savez(outPath, R=filteredColor, G=filteredColor,B=filteredColor, S=self.S,  colored_point_cloud=filteredPcd, bbox = self.bbox, res = self.res)



    def savez(self, out_file):
        np.savez(out_file, R=self.R, G=self.G,B=self.B, S=self.S,  colored_point_cloud=self.pcd, bbox = self.bbox, res = self.res)



In [6]:

# the path of the original mesh
meshPath = Path("C:/Users/u0146408/Documents/Datasets/Shapenet/03001627/1b5e876f3559c231532a8e162f399205/models/model_normalized.obj")
targetFolder = Path("data/preparedData")
mesh = trimesh.load(meshPath)
subMeshes = mesh.dump()

file_name = meshPath.parent.parent.name
if not os.path.exists(targetFolder / file_name):
    os.mkdir(targetFolder / file_name)
out_file = targetFolder / file_name / '{}_voxelized_colored_point_cloud_res{}_points{}_bbox{}.npz'\
    .format(file_name, res, num_points, bbox_str)
print(out_file)

# check if the mesh multiple materials (sub meshes)
if isinstance(mesh, trimesh.Scene):
    pointArray = []
    i = 1
    for sub in subMeshes:
        colored_addition = np.hstack((sub.sample(num_points), np.ones((num_points,1)) * i))
        pointArray.append(colored_addition)
        i+=1

    pointArray = np.asarray(pointArray).reshape((-1,4))
    colored_point_cloud = pointArray[np.random.choice(len(pointArray), size=num_points, replace=False)]
else:
    colored_point_cloud = np.hstack((mesh.sample(num_points), np.ones((num_points,1))))    


# encode uncolorized, complete shape of object (at inference time obtained from IF-Nets surface reconstruction)
# encoding is done by sampling a pointcloud and voxelizing it (into discrete grid for 3D CNN usage)
full_shape = as_mesh(trimesh.load(meshPath))
shape_point_cloud = full_shape.sample(num_points)
S = np.zeros(len(grid_points), dtype=np.int8)

_, idx = kdtree.query(shape_point_cloud)
S[idx] = 1

R = G = B =  colored_point_cloud[:,3]
colored_point_cloud = colored_point_cloud[:,:3]

np.savez(out_file, R=R, G=G,B=B, S=S,  colored_point_cloud=colored_point_cloud[:,:3], bbox = bbox, res = res)


data\preparedData\1b5e876f3559c231532a8e162f399205\1b5e876f3559c231532a8e162f399205_voxelized_colored_point_cloud_res128_points100000_bbox[-7, 7, -1, 20, -7, 7].npz


In [21]:
array = shoot_holes(colored_point_cloud, 4, 2e-2)
print(len(colored_point_cloud))
print(len(G))

100000
100000


In [20]:
print(array.shape)
print(array)

(8000,)
[    1    54    57 ... 99983 99986 99987]


In [28]:
filteredArray = np.delete(colored_point_cloud,array,0)
print(colored_point_cloud)
print(filteredArray)

[[ 0.06068785 -0.270231    0.14967037]
 [-0.18687995  0.29920344  0.17463423]
 [ 0.01124351 -0.007079    0.0685923 ]
 ...
 [-0.187144   -0.0927113  -0.23346742]
 [ 0.1096477  -0.03310676 -0.247366  ]
 [ 0.1339003  -0.12592858  0.11481341]]
[[ 0.06068785 -0.270231    0.14967037]
 [ 0.01124351 -0.007079    0.0685923 ]
 [-0.17196148 -0.32955715  0.157627  ]
 ...
 [-0.187144   -0.0927113  -0.23346742]
 [ 0.1096477  -0.03310676 -0.247366  ]
 [ 0.1339003  -0.12592858  0.11481341]]
